In [4]:
#library Import
import numpy as np
import pandas as pd
import pickle
#import flask
from flask import Flask, request, jsonify, make_response
from flask_restful import reqparse, abort, Api, Resource
from sklearn.preprocessing import OneHotEncoder

In [2]:
app = Flask(__name__)
app = Api(app = flask_app, 
     version = "1.0", 
     title = "Promise Date Prediction", 
     description = "Promise Date Prediction")
name_space = app.namespace('prediction', description='Prediction APIs')

NameError: name 'flask_app' is not defined

In [13]:
df=pd.read_excel("C:\\Users\\manohara.a\\OneDrive - SLK SOFTWARE\\pdsl\\test_prediction.xlsx")

In [14]:
#ip_path="C:\Users\manohara.a\Downloads\emerson\pdsl"
#Load data and filter only completed orders


def data_preprocessing(request_data):
    data1=request_data
    #data1=pd.read_excel("pdsl/test_prediction.xlsx")
    #data1=data1.loc[data1["FLOW_STATUS_CODE"]=="CLOSED"]
    drop_columns=['MODEL_STRING','ECCN','END_USE','SIC_CODE','HTSN','EXPORT_FLAG','REASON_FOR_SUSPICION','SHIP_SET','HOLD_APPLIED_BY','HOLD_RELEASED_BY']
    data=data1.drop(drop_columns,axis=1)
    data=data[data['PROMISE_DATE'].notnull()].drop_duplicates()
    data["hold_applied"]=data["HOLD_NAME"].apply(lambda x: '1' if x!="" else '0')
#holds_data
    holds_data=data.drop(['HOLD_NAME','HOLD_CREATION_DATE','HOLD_RELEASE_DATE','DELIVERY_ID','REQUESTED_QUANTITY','PICKED_QUANTITY'],axis=1).drop_duplicates()
    joindata=holds_data.groupby(["LINE_ID"]).agg({"SHIPPED_QUANTITY":'sum',"ORDERED_QUANTITY":'sum'})
    main_data=holds_data.drop(["SHIPPED_QUANTITY","ORDERED_QUANTITY"],axis=1).drop_duplicates()
    final_data=main_data.merge(joindata,on='LINE_ID',how='left')
    final_data["PROMISE_DATE"]=pd.to_datetime(final_data["PROMISE_DATE"])
    final_data["ULTIMATE_DROPOFF_DATE"]=pd.to_datetime(final_data["ULTIMATE_DROPOFF_DATE"])
    final_data["ORDERED_DATE"]=pd.to_datetime(final_data["ORDERED_DATE"])
    #final_data["label"]=(final_data['ULTIMATE_DROPOFF_DATE']-final_data['ORDERED_DATE'])/np.timedelta64(1, 'D')
    join_data=final_data[["LINE_ID","ORDERED_DATE","PROMISE_DATE",'OE_TRANSACTION_TYPES','ORDERED_ITEM','ULTIMATE_DEST','END_USER_SPECIFIC_PRODUCTS']]
    final_data = final_data[final_data.columns.drop(list(final_data.filter(regex='DATE')))]
    final_data = final_data.drop(["ORDER_NUMBER","ORG_ID","HEADER_ID","FLOW_STATUS_CODE","SOURCE_TYPE_CODE","HEADER_ID_1"],axis=1)
    final_data= final_data.apply(lambda x:x.fillna(x.value_counts().index[0]))
    #X=pd.get_dummies(data=final_data, columns=['OE_TRANSACTION_TYPES', 'ITEM_TYPE_CODE','ORDERED_ITEM','ULTIMATE_DEST','END_USER_SPECIFIC_PRODUCTS','SHIPPING_METHOD_CODE','SHIP_TO_COUNTRY','SHIP_FROM_COUNTRY','SHIP_FROM_COUNTRY',])
    enc=pickle.load(open("C:\\Users\\manohara.a\\OneDrive - SLK SOFTWARE\\pdsl\\pdsl_enc.pkl","rb"))
    encoded_data=enc.transform(final_data[['OE_TRANSACTION_TYPES', 'ITEM_TYPE_CODE','ORDERED_ITEM','ULTIMATE_DEST','END_USER_SPECIFIC_PRODUCTS','SHIPPING_METHOD_CODE','SHIP_TO_COUNTRY','SHIP_FROM_COUNTRY','SHIP_FROM_COUNTRY']]).toarray()
    encoded_data=pd.DataFrame(encoded_data,columns=enc.get_feature_names())
    X=final_data.join(encoded_data).drop(['OE_TRANSACTION_TYPES', 'ITEM_TYPE_CODE','ORDERED_ITEM','ULTIMATE_DEST','END_USER_SPECIFIC_PRODUCTS','SHIPPING_METHOD_CODE','SHIP_TO_COUNTRY','SHIP_FROM_COUNTRY','SHIP_FROM_COUNTRY'],axis=1)
    #y=dummy_data.loc[:, dummy_data.columns == 'label']
    #X=dummy_data.loc[:, dummy_data.columns != 'label']
    #X=dummy_data
    return X,join_data
    

In [15]:
X,join_data=data_preprocessing(df)

In [16]:
X.shape

(14, 3603)

In [17]:
reg=pickle.load(open("C:\\Users\\manohara.a\\OneDrive - SLK SOFTWARE\\pdsl\\pdsl_dtree.pkl","rb"))


In [18]:
pred=reg.predict(X)

In [45]:
join_data['pred']=pred
temp = join_data['pred'].apply(lambda x: pd.Timedelta(x, unit='D'))
join_data["predicted_date"]=join_data['ORDERED_DATE']+temp
join_data["Expected_Delay"]=(join_data['predicted_date']-join_data['PROMISE_DATE'])

In [49]:
join_data.sort_values(['Expected_Delay'])

LINE_ID ORDERED_DATE PROMISE_DATE            OE_TRANSACTION_TYPES  \
10  1219136212   2019-09-30   2020-01-13     Order_TD_US EndUse_USA_VLVS   
2   1218963619   2019-10-01   2019-12-16     Order_Intercompany USA_VLVS   
5   1219099223   2019-10-01   2019-11-08  Order_TD_NonUS EndUse_USA_VLVS   
9   1219990414   2019-10-01   2019-10-10     Order_TD_US EndUse_USA_VLVS   
11  1219136444   2019-09-30   2019-10-03     Order_TD_US EndUse_USA_VLVS   
12  1219136448   2019-09-30   2019-10-03     Order_TD_US EndUse_USA_VLVS   
13  1219136432   2019-09-30   2019-10-03     Order_TD_US EndUse_USA_VLVS   
0   1218963611   2019-10-01   2019-11-01     Order_Intercompany USA_VLVS   
6   1219099220   2019-10-01   2019-10-03  Order_TD_NonUS EndUse_USA_VLVS   
7   1219099239   2019-10-01   2019-10-03  Order_TD_NonUS EndUse_USA_VLVS   
3   1219099210   2019-10-01   2019-10-03  Order_TD_NonUS EndUse_USA_VLVS   
4   1219099219   2019-10-01   2019-10-03  Order_TD_NonUS EndUse_USA_VLVS   
1   1218963615   2019-10-01   2019-10-02     Order_Intercompany USA_VLVS   
8   1219135296   2019-10-01   2019-10-03     Order_TD_US EndUse_USA_VLVS   

   ORDERED_ITEM         ULTIMATE_DEST  \
10  37B1089X042         United States   
2   GE00185X012  United Arab Emirates   
5   22A3259X312                Canada   
9   1V6577X0042         United States   
11  RGASKETX182         United States   
12  RGASKETX172         United States   
13  RGASKETX442         United States   
0   1F548214012  United Arab Emirates   
6   13A4936X012                Canada   
7   22A3256X012                Canada   
3   RPACKX00022           Puerto Rico   
4   RGASKETX252           Puerto Rico   
1   0D005916012  United Arab Emirates   
8   R667X000502         United States   

                           END_USER_SPECIFIC_PRODUCTS  pred predicted_date  \
10                                     Domestic Order  99.0     2020-01-07   
2                                  Stock Order IR/ISO  76.0     2019-12-16   
5                             Natural Gas Natural Gas  38.0     2019-11-08   
9                                      Domestic Order   9.0     2019-10-10   
11                                     Domestic Order   4.0     2019-10-04   
12                                     Domestic Order   4.0     2019-10-04   
13                                     Domestic Order   4.0     2019-10-04   
0                                  Stock Order IR/ISO  41.0     2019-11-11   
6                             Natural Gas Natural Gas  13.0     2019-10-14   
7                             Natural Gas Natural Gas  13.0     2019-10-14   
3   Spare parts for control valve handlingprocess ...  28.0     2019-10-29   
4   Spare parts for control valve handlingprocess ...  28.0     2019-10-29   
1                                  Stock Order IR/ISO  41.0     2019-11-11   
8                                      Domestic Order   7.0     2019-10-08   

                 Expected_Delay  
10  -6 days +00:00:00.000000000  
2     0 days 00:00:00.000000000  
5     0 days 00:00:00.000000000  
9     0 days 00:00:00.000000000  
11    1 days 00:00:00.000000000  
12    1 days 00:00:00.000000000  
13    1 days 00:00:00.000000000  
0    10 days 00:00:00.000000000  
6    11 days 00:00:00.000000000  
7    11 days 00:00:00.000000000  
3    26 days 00:00:00.000000000  
4    26 days 00:00:00.000000000  
1    40 days 00:00:00.000000000  
8     5 days 00:00:00.000000000

In [47]:
join_data["Expected_Delay"]=join_data["Expected_Delay"].astype(str)


In [48]:
join_data.dtypes

LINE_ID                                int64
ORDERED_DATE                  datetime64[ns]
PROMISE_DATE                  datetime64[ns]
OE_TRANSACTION_TYPES                  object
ORDERED_ITEM                          object
ULTIMATE_DEST                         object
END_USER_SPECIFIC_PRODUCTS            object
pred                                 float64
predicted_date                datetime64[ns]
Expected_Delay                        object
dtype: object

In [44]:
join_data.to_excel("C:\\Users\\manohara.a\\OneDrive - SLK SOFTWARE\\pdsl\\model_prediction.xlsx")

In [23]:
test=join_data[:10]

In [32]:
jsos_df=test.to_json(orient='records')

In [33]:
type(jsos_df)

str

In [34]:
jsos_df

'[{"LINE_ID":1218963611,"ORDERED_DATE":1569888000000,"PROMISE_DATE":1572566400000,"OE_TRANSACTION_TYPES":"Order_Intercompany USA_VLVS","ORDERED_ITEM":"1F548214012","ULTIMATE_DEST":"United Arab Emirates","END_USER_SPECIFIC_PRODUCTS":"Stock Order IR\\/ISO","pred":41.0,"predicted_date":1573430400000,"Expected_Delay":864000000},{"LINE_ID":1218963615,"ORDERED_DATE":1569888000000,"PROMISE_DATE":1569974400000,"OE_TRANSACTION_TYPES":"Order_Intercompany USA_VLVS","ORDERED_ITEM":"0D005916012","ULTIMATE_DEST":"United Arab Emirates","END_USER_SPECIFIC_PRODUCTS":"Stock Order IR\\/ISO","pred":41.0,"predicted_date":1573430400000,"Expected_Delay":3456000000},{"LINE_ID":1218963619,"ORDERED_DATE":1569888000000,"PROMISE_DATE":1576454400000,"OE_TRANSACTION_TYPES":"Order_Intercompany USA_VLVS","ORDERED_ITEM":"GE00185X012","ULTIMATE_DEST":"United Arab Emirates","END_USER_SPECIFIC_PRODUCTS":"Stock Order IR\\/ISO","pred":76.0,"predicted_date":1576454400000,"Expected_Delay":0},{"LINE_ID":1219099210,"ORDERED_DA

In [35]:
df = pd.io.json.json_normalize(jsos_df, 'data')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


TypeError: string indices must be integers

In [ ]:
jsos_df.prin